In [1]:
# import the required libraries
import glob
import os
import posixpath
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torch.utils.data import Dataset, DataLoader
import cv2

In [2]:
def printPoints(image_file, points):
    
    points = points.reshape((int(points.shape[0]/2),2))
    
    ori_image = cv2.imread(image_file)

    for (x, y) in points:
        cv2.circle(ori_image, (int(x),int(y)), 0, (0, 255, 0), -1)

    plt.imshow(cv2.cvtColor(ori_image, cv2.COLOR_BGR2RGB))
    plt.show()

In [4]:
def datasetToCSV(trainPath, testPath):
    
    train_path =posixpath.join(trainPath, "*.png")
    train_images= np.array(glob.glob(train_path, ))
    for i, image in enumerate(train_images):
        train_images[i] = train_images[i].replace("\\", "/")

    train_landmarks = np.array([])
    for i, image_file in enumerate(train_images):
        points_file= image_file.replace("png", "pts")

        points=np.genfromtxt(points_file, delimiter=' ')

        points = points.flatten()
        if(i == 0):
            train_landmarks = np.hstack((train_landmarks,points))
        else:
            train_landmarks = np.vstack((train_landmarks,points))
    
    test_path =posixpath.join(testPath, "*.png")
    test_images= np.array(glob.glob(test_path, ))
    for i, image in enumerate(test_images):
        test_images[i] = test_images[i].replace("\\", "/")
    test_landmarks = np.array([])
    for i, image_file in enumerate(test_images):
        points_file= image_file.replace("png", "pts")

        points=np.genfromtxt(points_file, delimiter=' ')

        points = points.flatten()
        if(i == 0):
            test_landmarks = np.hstack((test_landmarks,points))
        else:
            test_landmarks = np.vstack((test_landmarks,points))
            
    
    land_cols = []
    for i in range(136):
        land_cols.append("Land{}".format(i))
        
    for i, image in enumerate(test_images):
        test_images[i] = test_images[i].split('/')[-1]
        
    for i, image in enumerate(train_images):
        train_images[i] = train_images[i].split('/')[-1]
        
    train_path_df = pd.DataFrame(train_images, columns = ["Path"])
    train_land_df = pd.DataFrame(train_landmarks, columns = land_cols)
    
    train_df = pd.concat([train_path_df, train_land_df], axis=1)
    
    test_path_df = pd.DataFrame(test_images, columns = ["Path"])
    test_land_df = pd.DataFrame(test_landmarks, columns = land_cols)
    
    test_df = pd.concat([test_path_df, test_land_df], axis=1)
    
    train_df.to_csv('data/train.csv', index=False)
    test_df.to_csv('data/test.csv', index=False)


    return train_images, train_landmarks, test_images, test_landmarks
    
train_images, train_landmarks, test_images, test_landmarks = datasetToCSV("bbimages224/train/", "bbimages224/test/")

In [5]:
class LandmarkDataset(Dataset):
    """300W Landmark dataset."""

    def __init__(self, csv_file, img_dir):
        """
        Args:
            csv_file: Path to the csv file with image paths and landmarks.
            img_dir: Directory with all the images.
        """
        self.landmarks = pd.read_csv(csv_file)
        self.img_dir = img_dir

    def __len__(self):
        return len(self.landmarks)

    def __getitem__(self, index):
        image_path = os.path.join(self.img_dir, self.landmarks.iloc[index, 0])
        
        image = mpimg.imread(image_path)
        
        # if image has an alpha color channel, get rid of it
        if(image.shape[2] == 4):
            image = image[:,:,0:3]
        
        landmarks = self.landmarks.iloc[index, 1:].to_numpy()
        landmarks = landmarks.astype('float').reshape((int(landmarks.shape[0]/2),2))
        sample = {'image': image, 'keypoints': key_pts}

        return sample

In [7]:
# Construct the dataset
landmark_dataset = LandmarkDataset(csv_file='data/train.csv', img_dir='bbimages224/train/')

# print some stats about the dataset
print('Length of dataset: ', len(landmark_dataset))

Length of dataset:  480


In [10]:
k = np.array([])
l = np.array([1,2])
k = np.hstack((k,l))
print(k)
k = np.vstack((k,l))
print(k)

[1. 2.]
[[1. 2.]
 [1. 2.]]


In [13]:
h = np.array([[1.2, 1.5, 1.49, 1.9],[1.2, 1.5, 1.49, 1.9]])
print(np.around(h))

[[1. 2. 1. 2.]
 [1. 2. 1. 2.]]


In [16]:
np.sqrt(np.sum(np.square(np.around(h))))

4.47213595499958

In [11]:
true = np.array([[ 47.,35.],
 [ 68.,35.],
 [ 91.,33.],
 [108.,30.],
 [ 91.,59.]])

pred = np.array([[ 50.,36.],
 [ 67.,34.],
 [ 91.,33.],
 [105.,29.],
 [ 91.,61.]])

d = np.sqrt(128*128)
nme = 0
for i in range(len(true)):
    nme+=(np.sum(np.square(true[i]-pred[i]))/d)

nme = nme/5
nme

0.040625

In [8]:
len(true)

5